
# Mercari Price Suggestion Algorithm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import category_encoders as ce

In [ ]:
# Cài đặt 7zip và giải nén tất cả các files
!apt install --assume-yes p7zip-full
!7z x ../input/mercari-price-suggestion-challenge/train.tsv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/test.tsv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/test_stg2.tsv.zip -y
!7z x ../input/mercari-price-suggestion-challenge/sample_submission.csv.7z -y
!7z x ../input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip -y

In [ ]:
#đọc dữ liệu
train = pd.read_csv("train.tsv",sep = '\t')

In [ ]:
#kích thước của mảng
train.shape

In [ ]:
train.head()

## Exploring Price

Hypothesis: Price will follow a log-normal distribution

In [ ]:
train['log_price'] = np.log1p(train.price)

In [ ]:
from scipy.stats import norm

#khởi tạo figure và axes
fig, axes = plt.subplots(1, 3, figsize=(10,4))

axes[0].set_title('Price')
sns.distplot(train.price, ax=axes[0], kde=False)
axes[0].grid()

axes[1].set_title('Price < 100')
sns.distplot(train.price[train.price<100], ax=axes[1], kde=False)
axes[1].grid()

axes[2].set_title('log(Price + 1)')
sns.distplot(train.log_price, ax=axes[2], fit=norm, kde=False)
axes[2].set_xticks(range(0,9))
axes[2].grid()

fig.tight_layout()

Ta thấy giá tuân theo Log-Normal Distribution với một số tăng đột biến xung quanh các số tròn như 10, 25, 30 và 40 đô la. 

## Exploring Data Attributes

In [ ]:
def plot_distribution_and_violin(variable):
    fig, axes = plt.subplots(2,1,figsize=(5,6), sharex=True)
    axes[0].set_title(variable)
    sns.countplot(x=variable, data=train, palette="ch:.25", color="c", ax=axes[0])
    sns.violinplot(x=variable, y='log_price', palette="ch:.25", data=train, ax=axes[1])
    fig.tight_layout()

Các mặt hàng trong tình trạng tốt hơn sẽ có giá trung bình cao hơn

In [ ]:
plot_distribution_and_violin('item_condition_id')

Các mặt hàng mà người bán thanh toán phí vận chuyển (1) sẽ có giá trung bình cao hơn so với nơi người mua thanh toán tiền vận chuyển (0)

In [ ]:
plot_distribution_and_violin('shipping')
print('shipping - 1 if shipping fee is paid by seller and 0 by buyer')

In [ ]:
avg_log_price = train.groupby('shipping')['log_price'].mean()
diff = np.expm1(avg_log_price[0]) - np.expm1(avg_log_price[1])
print(f'Phí vận chuyển do người mua thanh toán ${diff:.3} lớn hơn phí vận chuyển do người bán thanh toán')

Trên thực tế vận chuyển do người mua thanh toán đã lớn hơn. Có thể điều đó có nghĩa là "miễn phí vận chuyển" được sử dụng phổ biến hơn cho các mặt hàng rẻ hơn.

#### Decompose category

Danh mục có thể được phân chia thành ba cấp độ (ví dụ: Nam / Áo sơ mi / Áo phông). Trong dữ liệu, chúng được phân định bằng dấu gạch chéo về phía trước. Một số loại cấp độ thứ ba có dấu gạch chéo trong tên, nhưng vẫn đáng tin cậy để phân chia ở hai dấu gạch chéo đầu tiên.

In [ ]:
train[train.category_name.str.count('/')!=2].groupby('category_name').count()#sort_values(ascending=False)

In [ ]:
def transform_split_category_name(df):
    category_split = df['category_name'].str.split(r'/', n=2, expand=True)
    for i in [0,1,2]:
        df['cat_level_' + str(i)] = category_split[i]
    return df

In [ ]:
train = transform_split_category_name(train)

In [ ]:
train.head()

#### Analyze Description


In [ ]:
gb = train.groupby('item_description')['train_id'].count()
gb[gb>500].sort_values(ascending=False)

Có 82.489 mục có 'Chưa có mô tả' có thể được coi là một giá trị bị thiếu, nhưng các cụm từ ngắn rất phổ biến khác cũng thường bị trùng lặp.

Độ dài của mô tả có thể có một số sức mạnh dự đoán.

## Analyze Brands

In [ ]:
def transform_missing_brand(df):
    df['missing_brand'] = df.brand_name.isna()
    return df

In [ ]:
train = transform_missing_brand(train)

In [ ]:
plot_distribution_and_violin('missing_brand')

In [ ]:
# Các thương hiệu phổ biến nhất
train.groupby('brand_name')['train_id'].count().sort_values(ascending=False).head(40)

# Baseline Models

Đây là mô hình cơ sở dự đoán dựa trên giá trung bình của một trường

In [ ]:
class baseline_model():
    
    def fit(self):
        pass
    
    def predict(self, X):
        return np.zeros(shape=len(X))

In [ ]:
m = baseline_model()
m.fit()
m.predict([[1,2,3],
           [4,5,6]])

## Test Train Split

In [ ]:
X = train[[col for col in train.columns if col not in ['price', 'log_price']]]
y = train['log_price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Initial Model

- Hồi quy tuyến tính
- Nhắm mục tiêu các danh mục được mã hóa
- Tên_mục tiêu được mã hoá
- Tình trạng mặt hàng
- Đang chuyển hàng

In [ ]:
# Select necessary columns
X_train = X_train[['item_condition_id', 'category_name', 'brand_name', 'shipping']]
X_test = X_test[['item_condition_id', 'category_name', 'brand_name', 'shipping']]

In [ ]:
te = ce.TargetEncoder()
te.fit(X_train, y_train)
X_train = te.transform(X_train)
X_test = te.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
predictions = lr.predict(X_test)

In [ ]:
predictions

In [ ]:
sns.jointplot(predictions, y_test, kind="hex")

In [ ]:
kaggle_test = pd.read_csv("test_stg2.tsv",sep = '\t')

In [ ]:
kaggle_test = transform_split_category_name(kaggle_test)
X_kaggle_test = kaggle_test
X_kaggle_test = X_kaggle_test[['item_condition_id', 'category_name', 'brand_name', 'shipping']]
X_kaggle_test = te.transform(X_kaggle_test)

In [ ]:
X_kaggle_test.head()

In [ ]:
output = lr.predict(X_kaggle_test)

In [ ]:
submission = np.expm1(output)

In [ ]:
submission_df = pd.DataFrame({'test_id': kaggle_test.test_id, 'price': submission})

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df.shape

Ideas:

- Giảm kích thước của cột thương hiệu bằng cách mã hóa thương hiệu tương tự thành cùng một giá trị (gucci, louis)> thương hiệu_1 (adidas, nike)> thương hiệu_2
- Tính năng (nhị phân) là thương hiệu trong tiêu đề
- Xác định một số n-gam thay đổi giá trị ('không khói', 'hư hỏng nhẹ')
- Xóa thẻ từ? Có thể là thách thức ...
- Tìm lỗi chính tả trong brand_name (có thể không nhiều vì trường tự động điền)